In [1]:
import numpy as np
import forward as fwd
import truncatedfa as trfwd
import os
import pandas as pd
import re
from scipy import stats

In [4]:
# result_dir = '/Users/kwinner/Work/Data/Results/2018-08-14 20:23:31.654124 "latentcounts"'
result_dir = '/Users/kwinner/Work/Data/Results/2018-09-04 11:48:47.693671 "latentcounts"'

In [5]:
def load_results(result_dir):
    result_filename = os.path.join(result_dir, 'results.df')

    results = pd.read_table(result_filename, sep=' ')

    return(results)

results = load_results(result_dir)

In [6]:
def str2mat(str):
    # remove leading/trailing brackets
    str = re.search('(?=\\[*)[^\\[].*[^\\]](?=\\]*)', str).group(0)

    if '];[' in str: # 2D array
        # convert to numeric list
        mat = np.array([[float(i) for i in row.split(',')] for row in str.split('];[')])
    else:
        mat = np.array([float(i) for i in str.split(',')])

    return(mat)

In [26]:
y = str2mat(results.y[1])

In [8]:
results.columns

Index(['rho.gen', 'method', 'rt', 'nll', 'n.iters', 'fit.lambda', 'fit.gamma',
       'fit.rho', 'fit.iota', 'x.lambda', 'x.gamma', 'x.rho', 'x.iota', 'y',
       'K', 'n.attempts'],
      dtype='object')

In [43]:
x = np.array(results.loc[1,'fit.lambda':'fit.iota'], np.float64)

lmbda = np.concatenate((np.array([results.loc[1,'fit.lambda']]), results.loc[1, 'fit.iota'] * np.ones(y.shape[1] - 1))).reshape((-1,1))
delta = results.loc[1, 'fit.gamma'] * np.ones(y.shape[1] - 1).reshape((-1,1))
rho = results.loc[1, 'fit.rho'] * np.ones(y.shape[1])

print(lmbda)
print(delta)
print(rho)

[[ 1.52242452]
 [ 0.77424081]
 [ 0.77424081]
 [ 0.77424081]
 [ 0.77424081]]
[[ 0.60443202]
 [ 0.60443202]
 [ 0.60443202]
 [ 0.60443202]]
[ 0.9991021  0.9991021  0.9991021  0.9991021  0.9991021]


In [41]:
arrival_dist = stats.poisson
# branching_fn = trfwd.binomial_branching
branching_fn = trfwd.poisson_branching

In [47]:
alpha, logz = trfwd.truncated_forward(arrival_dist, lmbda, branching_fn, delta, rho, y[0,], 100)
alpha, logz2 = trfwd.truncated_forward(arrival_dist, lmbda, branching_fn, delta, rho, y[1,], 100)
print(logz + logz2)

-15.3775234738


In [38]:
y

array([[ 3.,  3.,  2.,  4.,  4.],
       [ 1.,  1.,  2.,  1.,  0.]])

In [39]:
y[0,]

array([ 3.,  3.,  2.,  4.,  4.])